# Dévéloppement d'une API ou d'une application web en Python

## FastAPI

FastAPI est un framework qui sert à développer des API REST en Python. Il autorise la programmation asynchrone.

On commence notre code par importer la classe ``fastapi.FastAPI`` et définir notre application comme une instance de cette classe.

Deux notions importantes qui permettent à FastAPI de traiter les requêtes sont: le chemin et l'opération. 
- Le chemin fait référence à la dernière partie de l'URL à partir du premier "/". Par exemple, dans l'URL "https://www.sspcloud.fr/formation", le chemin est ```/formation```.

- l'opération est une méthode HTTP: POST (création de données), GET (lecture), PUT (modifier), DELETE (supprimer)...

Dans le protocole HTTP, on communique avec chaque chemin en utilisant une ou plusieurs de ces opérations. Ainsi, dans notre API, on va définir un path et une opération pour chaque fonction qu'on va proposer.

Pour ce faire, on précède chaque fonction par l'expression suivante: ```@app_name.operation(path)```.

- Dans le code suivant, la fonction **root** sert comme un accueil pour l'API et elle retourne un message d'accueil. Elle est précédée par ```@app.get('/')```. Cette expression indique à FastAPI que la fonction "root" est chargée des requêtes qui vont au chemin "/", en utilisant l'opération "get".

- La fonction **predict** prend en entrée un fichier image et donne en retour la classe de l'image ainsi que la probabilité de la prédiction. On utilise la syntaxe ``async`` et ``await`` pour qu'elle soit asynchrone. Cela indique à Python d'éxecuter d'autres tâches en attendant que les données soient envoyées du client au serveur à travers le réseau. Cette méthode permet au serveur d'optimiser son temps de réponse.

- On définit également une fonction **details** sur le chemin ``/model/{info}`` et en utilisant l'opération ``get`` pour afficher des informations sur le modèle. Le paramètre ``info`` est appelé paramètre de chemin et il est à préciser par le client dans sa requête. Par exemple, le chemin ``/model/accuracy`` retourne la précision du modèle. 
On ajoute aussi un paramètre de requête de type entier appelé ``n``. Ce paramètre est un entier qui permet de préciser le nombre de chiffres après la virgule dans la précision. On définit ce paramètre dans la requête comme dans l'exemple suivant: ``model/accuracy?n=1``.

FastAPI génère une documentation et une interface utilisateur pour l'API automatiquement en utilisant ``OpenAPI``. Cette interface est disponible dans le chemin ``/docs``. Vous pouvez consulter une instance de l'API créée dans ce tutoriel [en cliquant ici](https://fastapi-tuto.lab.sspcloud.fr/docs).

In [ ]:
#install dependencies
!pip install fastapi
!pip install uvicorn
!pip install python-multipart

***Fichier main de l'API***

In [ ]:
from io import BytesIO
from fastapi import FastAPI, File, UploadFile
from utils import load_device, import_model, predict, is_image_file
from PIL import Image

 
app = FastAPI()

def read_image(file):
    img = Image.open(BytesIO(file))
    return img

device = load_device()
model = import_model(bucket="mbenxsalha", key="diffusion/state_dict.pickle", device=device)

#url: localhost:8000
@app.get("/")
def root():
    return {"message": "Welcome to Image Classification FastAPI"}

#url: localhost:8000/model
@app.get("/model/{info}")
def details(info:str, n:int=2):
    accuracy = 99.2511111
    if info == 'model':
        return {'model': 'ResNet18'}
    elif info == 'dataset':
        return {'dataset url': "https://www.kaggle.com/datasets/carlosrunner/pizza-not-pizza"}
    elif info == 'accuracy':
        formatted_accuracy = int((10**n)*accuracy)/(10**n)
        return {'accuracy': '{}'.format(formatted_accuracy)}
    else:    
        return '{} is not available'.format(info)

#url: localhost:8000/predict
@app.post("/predict")
async def predict_api(file: UploadFile = File(...)):
    if not is_image_file(file.filename):
        return "file must have image format"
    img = read_image(await file.read())
    preds = predict(img, model, device)
    return preds



**Pour lancer l'API, on utilise "uvicorn" qui permet d'exécuter un code asynchrone sur Python.**
Dans le code suivant, ``main`` fait référence au fichier main.py qui contient le même code que la cellule précédente.

La tag ``--reload`` permet de relancer l'API automatiquement si le code est modifié.

In [ ]:
!uvicorn main_fastapi:app --reload --host=0.0.0.0 --proxy-headers

Le notebook ``examples-fastapi.ipynb`` illustre comment communiquer et envoyer des requêtes à l'instance que vous venez de créer.

## Flask

La répertoire doit contenir deux dossiers: "templates" et "static".

- Le dossier ``templates`` contient les templates html que flask utilisera pour construire les pages web. 

- Le dossier ``static`` contient des fichiers d'affichage (image, ficher css..)

On commence par importer la classe Flask et définir une instance qui sera notre application.
Le site contient deux pages: une page d'accueil et une page pour afficher les résulats.
Ainsi, on va définir une fonction et un template html pour chaque page.
Le code de chaque fonction est précédée par la synatxe ```@app.route(chemin:str, methods:list)```. Cette expression indique à Flask que cette fonction est chargée de requêtes qui vont au chemin indiqué et qui utilisent la liste de méthodes indiquée.

Chaque fonction donne en sortie ```render_template('page.html')``` qui indique à Flask la template à interpréter pour cette fonction.

La fonction **predict** retourne également des variables (pred, user_image) qui serviront à définir les valeurs des variables output et user_image dans le fichier predict.html.



In [ ]:
#install flask
!pip install flask

***Fichier main de l'application Flask***

In [ ]:
from io import BytesIO
from flask import Flask, render_template, request
from utils import load_device, import_model, predict, load_image
from PIL import Image
import os

def read_image(file):
    img = Image.open(BytesIO(file))
    return img

app = Flask(__name__)

device = load_device()
model = import_model(bucket="mbenxsalha", key="diffusion/state_dict.pickle", device=device)


@app.route("/", methods=["GET", "POST"])
def home():
    return render_template('home.html')

@app.route("/predict", methods=["GET", "POST"])
def predict_flask():
    if request.method == "POST":
        file = request.files['file']
        filename = file.filename
        file_path = os.path.join('static', filename)
        file.save(file_path)
        img = load_image(file_path)
        pred = predict(img, model, device)
    return render_template("predict.html", output=pred, user_image = file_path)


On exécute la commande suivante pour lancer l'application. ``main_flask`` fait référence au fichier ``main_flask.py``.

In [ ]:
!flask --app main_flask run --host=0.0.0.0

Vous pouvez consulter l'application que vous venez de créer en changeant le lien du notebook comme dans l'exemple suivant: 
- lien de notebook: https://user-username-239011-0.user.lab.sspcloud.fr/
- modifier le "-0" par "-user". Donc pour cet exemple, le lien de l'application est: https://user-username-239011-user.user.lab.sspcloud.fr/

Créer ce lien et donc communiquer avec l'application depuis l'extérieur est possible en autorisant l'activation d'un port de service personnalisé (via ``jupyter-python configurations Networking`` avant de lancer le notebook) et en réglant le paramètre ``custom service port`` sur lequel votre application s'exécutera.

## Streamlit

Streamlit est une bibliothèque Python qui permet de créer des application web sans avoir besoin de savoir le développement web. 

On écrit les différentes parties dans l'ordre que l'on souhaite s'afficher sur la page web. 
- ``st.title(title:str)`` permet d'afficher un titre
- ``st.write(text:str)`` permet d'écrire un texte
- ``st.file_uploader()`` permet d'ajouter un bouton ``upload`` pour importer des fichiers. Le fichier importé sera sous la forme de bytes.

Streamlit permet aussi de mettre des données en cache. Cela est utile pour enregistrer les résultats de fonctions qu'on utilise souvent avec les mêmes paramètres (``import_model`` par exemple). Donc, Streamlit va calculer la fonction seulement pour la première exécution. Cela est possible avec la synatxe ``@st.cache()`` qu'on écrit avant la fonction qu'on veut enregistrer.

In [ ]:
#install streamlit
!pip install streamlit

***Fichier utils de l'application streamlit***

In [ ]:
from PIL import Image
import torch
from model import ResNet
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import streamlit as st
import boto3
import pickle

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP']

#@st.cache()
def import_model(bucket, key="diffusion/state_dict.pickle", device=torch.device('cpu')):
    s3 = boto3.client('s3',endpoint_url='https://minio.lab.sspcloud.fr/')
    data = s3.get_object(Bucket=bucket, Key=key)
    state_dict = pickle.loads(data['Body'].read())
    model = ResNet()
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return model

def load_image(file):
    img = Image.open(file).convert("RGB")
    return img

def predict(img, model, device):
    img = TF.to_tensor(img)
    img = TF.normalize(img, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    img = img.to(device)
    img = img.unsqueeze(0)
    preds = model(img)
    preds = F.softmax(preds, dim=1)
    if float(preds[0][0]) < float(preds[0][1]):
        results = "There is a ship!"
    else:
        results = "There is no ship!"
    return results

@st.cache()
def load_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    return device
@st.cache()
def is_image_file(filename):
  return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

***Fichier main de l'application Streamlit***

In [ ]:
import torch

device = load_device()
model = import_model(bucket="mbenxsalha", key="diffusion/state_dict.pickle", device=device)

st.title("Welcome to The Ship Detective!")
st.write("The image you upload will be fed to a Deep Neural Network in real-time to verify if there is a ship or not")
file = st.file_uploader("Upload an image")

if file:
    img = load_image(file)
    predictions = predict(img, model, device)
    st.title("Here is the image you uploaded")
    resized_image = img.resize((340, 340))
    st.image(resized_image)
    st.title("Prediction:")
    st.write(predictions)

In [ ]:
!streamlit run main_streamlit.py